In [2]:
# Reset and Restart 

import sys
import os
import numpy as np
import time
from imp import reload
import threading
import csv
from pathlib import Path
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
import pandas as pd
import glob
import skimage.measure as measure

import matplotlib.pyplot as plt

# Set the working directory to the AutomatedCE folder so we have access to L1-L4 modules
resp = os.getcwd()
if 'testing' in resp[-7:]:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
    print(f"new directory is: {os.getcwd()}")
import testing.inputs  
reload(testing.inputs)
from testing.inputs import *
from L4 import DisplayWindows
from L4 import FileIO

reload(DisplayWindows)

CONFIG_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Eclipse.cfg"
#CONFIG_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300.cfg"
from L3 import SystemsBuilder
reload(SystemsBuilder)

ce_system = SystemsBuilder.CESystem()

ce_system.load_config(CONFIG_PATH)
ce_system.open_controllers()
ce_system.startup_utilities()

%matplotlib qt

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Luke\\Desktop\\Barracuda\\AutomatedCE\\var\\Eclipse.cfg'

In [2]:
ce_system.inlet_rgb.turn_off_channel('r')

'R:On, G:Off, B:Off'

In [3]:
ce_system.inlet_rgb.turn_on_channel('B')

'R:On, G:Off, B:On'

In [3]:
from L4 import Focus
reload(Focus)

<module 'L4.Focus' from 'C:\\Users\\Luke\\Desktop\\Barracuda\\AutomatedCE\\L4\\Focus.py'>

# Automation Objects

In [33]:
# Set up the Automation Method & Template 
TEMPLATE_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\EclipseCellTemplate.txt"
#METHOD_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Brae\Cell.txt"

#TEMPLATE_PATH = r"C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\var\TE300CellTemplate.txt"
METHOD_PATH = r"C:\Users\Luke\Desktop\Barracuda\AutomatedCE\var\Brae\Fluorescein.txt"

from L4 import AutomatedControl
from L4 import Trajectory
reload(AutomatedControl)
reload(Trajectory)

auto_program = AutomatedControl.AutoRun(ce_system)
auto_program.set_template(TEMPLATE_PATH)
auto_program.add_method(METHOD_PATH)
auto_program.repetitions=3

WHASSUZ ['inject\tw_e0\t210\t7 mm\t0\t0\t0\t5\t\t0', 'standard 50to50\tw_e1\t240\t7 mm\t0\t0\t20\t60\t\t1']
ADDING
ADDING


In [27]:
ce_system.inlet_z.set_z(22)

In [69]:
# Start the Method
if not auto_program.continue_event.is_set():
    auto_program.continue_event.set()
else:
    auto_program.start_run()

In [26]:
# Stop the Method
auto_program.stop_run()

In [34]:
auto_program.start_run()

After special True
Time to wait is  5


C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\DisplayWindows.py:277: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  ax1.set_xlim(min(data['time_data']), max(data['time_data']))


After special True
Time to wait is  60
After special True
Time to wait is  5
After special True
Time to wait is  60


After special True
Time to wait is  5
After special True
Time to wait is  60


In [55]:
auto_program.data_dir

'.\\Data'

After special True
Time to wait is  5


C:\Users\Luke\Desktop\Barracuda\AutomatedCE\L4\DisplayWindows.py:277: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  ax1.set_xlim(min(data['time_data']), max(data['time_data']))


After special True
Time to wait is  180
After special True
Time to wait is  5


After special True
Time to wait is  180


After special True
Time to wait is  5
After special True
Time to wait is  180


After special True
Time to wait is  5
After special True
Time to wait is  180
After special True
Time to wait is  5


After special True
Time to wait is  180


After special True
Time to wait is  5


After special True
Time to wait is  180


In [54]:
ce_system.inlet_z.set_z(22)

In [31]:
ce_system.outlet_pressure.rinse_pressure()

In [32]:
ce_system.outlet_pressure.release()

In [42]:
name = get_value("Please enter the well name to move to (press enter to see a list): ", auto_program.template.wells.keys())
well = auto_program.template.wells[name]
ce_system.xy_stage.set_xy(well.xy)
resp = input("Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.")


Please enter the well name to move to (press enter to see a list): w_e7
Please move the capillary using the fine adjustment knobs, or the z-stage base so that capillary is centered over this well.


In [37]:
# CE Display
ce_gram = DisplayWindows.PLTCEDisplay(ce_system)
#ce_disp.stop()
#ce_disp.show()
ce_gram.start_live_view()
ce_system.detector.start()

In [68]:
# Microscope Display
cam = DisplayWindows.PLTMicroscopeDisplay(ce_system)
ce_system.camera.stop()
cam.show()
cam.live_image()

HEYYOO


In [40]:
ce_system.camera.set_exposure(200)

True

In [49]:
ce_system.xy_stage.controller.open()


# Set the Focus Plane for the cells

In [9]:
focus_plane = Focus.PlaneFocus(ce_system)

#focus_plane.get_three_points()


In [20]:
# run this three times
focus_plane.give_response()

In [21]:
focus_plane.find_a_plane()

True

In [22]:
with open('focus_save_me','w') as fout:
    fout.write(f'focus_plane,{focus_plane._plane_coefficients}')

In [10]:
with open('focus_save_me','r') as fin:
    focus_plane._plane_coefficients= eval(fin.readline().split('focus_plane,')[1])
    

In [16]:
focus_plane.get_plane_focus()

In [25]:
focus_search = Focus.FindFocus(ce_system)

# Set the Capillary Adjustments

In [40]:
# Move the objective to the height and run this cell to record it for next time
obj_height = ce_system.objective.read_z()
ce_system.objective.set_rel_z(+.05)

In [63]:
ce_system.inlet_z.set_rel_z(-0.03)


In [69]:
ce_system.inlet_z.set_z(5)

In [64]:
# and record this height for next time 
cap_height = ce_system.inlet_z.read_z()
cap_difference = obj_height - cap_height

In [65]:
with open('save_me','w') as fout:
    fout.write(f'cap_difference,{cap_difference}')

In [11]:
with open('save_me','r') as fin:
    cap_difference= float(fin.readline().split(',')[1])
    

# Single Cell Routine

In [12]:
rep = get_yes_no("Do you want to raise the objective?")
if rep:
    ce_system.objective_z.set_rel_z(+4)

-6.7641084

In [16]:
# on reruns, you just need to run this command when you have the cells in focus
resp = get_yes_no("are the cells in focus? (You may crush the capillary if not...)")
if resp:
    obj_height = ce_system.objective.read_z()
    print(obj_height)
    cap_z = get_cap_height(obj_height, cap_difference)
    z = ce_system.inlet_z.read_z()
    rel_z = cap_z-z
    ce_system.inlet_z.set_rel_z(rel_z)
else:
    print("Bring cells into focus.")

are the cells in focus? (You may crush the capillary if not...) y
-5.31


In [51]:
# General Injection Parameters
injection_time = 4 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 3 # in kilovolts
delay = 0 # inseconds

# Start the Single Cell Injection
import threading
las = ce_system.lysis_laser
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    las.laser_standby()
    time.sleep(1)
    las.laser_fire()
    while time.time()-start_time < injection_time:
        time.sleep(0.5)
        #ce_system.objective.set_rel_z(0.005)
        #ce_system.xy_stage.set_rel_x(0.001)
        #ce_system.lysis_laser.laser_fire()
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    

threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()

In [53]:
# Start the Method
def go_ahead():
    ce_system.inlet_z.set_rel_z(3)
    time.sleep(2)
    ce_system.objective.set_rel_z(-4)
    if not auto_program.continue_event.is_set():
        auto_program.continue_event.set()
    else:
        auto_program.start_run()
threading.Thread(target=go_ahead).start()

After special True
Time to wait is  3


After special True
Time to wait is  150


# Extra

In [35]:
ce_system.inlet_z.set_z(22)

In [4]:
las = ce_system.lysis_laser

In [9]:
las.stop()

In [10]:
las.laser_standby()

In [69]:
stop_flag = threading.Event()

def fire_on():
    while not stop_flag.is_set():
        las.laser_fire()
        time.sleep(0.5)
        
threading.Thread(target=fire_on).start()

In [70]:
stop_flag.set()

In [27]:
ce_system.camera.set_exposure(5)

True

In [10]:
# PARAMETERS
target_x = 871
target_y = 585
rgb_chnl = 'R'
fluor_channel = 1
wait = 0.25
exp = 500
_old_exp=100
_old_bin = 1
_old_channel = 0
bins = 1
conversion =0.0003125
obj_40x = 0.0001607717041800643


def pre_fluoresence(auto_shutter=False):
    # SNAP a FLUOR IMAGE
    # Set to new values
    ce_system.inlet_rgb.turn_off_channel(rgb_chnl)
    ce_system.camera.stop()
    ce_system.filter_wheel.set_channel(fluor_channel)
    if auto_shutter:
        ce_system.excitation_shutter.set_auto_on()
    #_old_exp = ce_system.camera.exposure
    #ce_system.camera.set_binning(bins)
    #_old_bin = ce_system.camera.bin_size
    time.sleep(wait)
    
def post_fluoresence(auto_shutter=False):
    # Return to old values
    ce_system.camera.set_exposure(_old_exp)
    #ce_system.camera.set_binning(_old_bin)
    ce_system.excitation_shutter.set_auto_off()
    ce_system.filter_wheel.set_channel(_old_channel)
    ce_system.inlet_rgb.turn_on_channel(rgb_chnl)
    ce_system.camera.continuous_snap()

def get_cap_height(obj_height, cap_difference):
    cap_height = obj_height - cap_difference
    return cap_height

def get_brightest_blobs(image):
    close_size=5
    open_size=5
    watershed_footprint = (5,5)
    # Make sure types are the same
    input_image = img_as_float(img)

    # Filter Image
    filtered_image = filters.median(input_image, behavior='ndimage')

    # Edge Detection
    edge_sobel = filters.sobel(filtered_image)

    # Threshold
    thresh = filters.threshold_otsu(filtered_image)
    binary_otsu = filtered_image > thresh

    # Binary Morphology Operations
    structure_element = morphology.disk(close_size)
    closed_image = morphology.binary_closing(binary_otsu, structure_element)
    structure_element = morphology.disk(open_size)
    opened_image = morphology.binary_opening(closed_image, structure_element)

    # Watershed
    distance = distance_transform_edt(opened_image)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
    markers = label(local_maxi)[0]
    labels = watershed(-distance, markers, mask=opened_image)
    
    # Labeling and Sorting
    regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))
    df = pd.DataFrame(regions)
    df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)
    
    return df

def move_to_blob(df):
    if df.shape[0] > 0:
        y=df['centroid-0'][0]*bins
        x=df['centroid-1'][0]*bins

        movex = (x-target_x)*conversion
        movey = (y-target_y)*conversion
        ce_system.xy_stage.set_rel_x(movex)
        ce_system.xy_stage.set_rel_y(-movey)
        
def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    start_time = time.time()
    ce_system.lysis_laser.laser_fire()
    ce_system.objective.set_rel_z(-0.010)

    while time.time()-start_time < injection_time:
        if time.time()-start_time < 3:
            ce_system.objective.set_rel_z(0.005)
            ce_system.xy_stage.set_rel_x(0.001)
            ce_system.lysis_laser.laser_fire()
        time.sleep(0.35)
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)

In [27]:
# Bring the Cells into focus
fluor_channel = 3
focus_plane.get_plane_focus()


In [28]:
data_dir = r"C:\Users\Luke\Documents\Brae\9-19-2020\images"
fluor_saver = ImageSaver(ce_system, folder_prefix='Picking', data_folder=data_dir)
focus_saver = ImageSaver(ce_system, folder_prefix='Focusing', data_folder=data_dir)
lysis_saver = ImageSaver(ce_system, folder_prefix='Lysis', data_folder=data_dir)

def automatic_sampling():
    # Bring the Cells into focus
    fluor_channel = 3
    focus_plane.get_plane_focus()
    time.sleep(3)
    
    # Test Alec's Focus and get data
    pre_fluoresence()
    focus_saver.add_callback()
    focus_search = Focus.FindFocus(ce_system)
    focus_search.search_step_global()
    focus_saver.remove_callback()
    post_fluoresence()
    focus_saver.save_image()
    
    # We don't Trust Alec's function yet
    fluor_channel = 3
    focus_plane.get_plane_focus()
    
    # Get Fluoresence Image for Best chance
    fluor_channel = 1
    pre_fluoresence()
    fluor_saver.add_callback()
    img = ce_system.camera.snap()
    fluor_saver.remove_callback()
    post_fluoresence()
    
    
    # Move to the brighest blob
    blobs = get_brightest_blobs(img)
    move_to_blob(blobs)
    
    
    # Bring the Cells into focus
    fluor_channel = 3
    focus_plane.get_plane_focus()
    lysis_saver.add_callback()
    time.sleep(0.5)
    
    # on reruns, you just need to run this command when you have the cells in focus
    resp= True
    if resp:
        obj_height = ce_system.objective.read_z()
        print(obj_height)
        cap_z = get_cap_height(obj_height, cap_difference)
        z = ce_system.inlet_z.read_z()
        rel_z = cap_z-z
        rel_z += 0.02
        ce_system.inlet_z.set_rel_z(rel_z)
    else:
        print("Bring cells into focus.")
        raise RuntimeError("Cells not in Focus")

    time.sleep(5)
    
    # Start the Single Cell Injection
    limited_fire(ce_system, gravity_drop, voltage_level, injection_time)
    
    # Prepare to start the method
    ce_system.inlet_z.set_rel_z(3)
    time.sleep(3)
    lysis_saver.remove_callback()
    lysis_saver.save_image()
    
    # Start the Method
    ce_system.objective.set_rel_z(-3)
    if not auto_program.continue_event.is_set():
        auto_program.continue_event.set()
    else:
        auto_program.start_run()

NameError: name 'ImageSaver' is not defined

In [ ]:
light = 

# TE300 Automation

In [11]:
 # PARAMETERS
target_x = 871
target_y = 585
rgb_chnl = 'B'
fluor_channel = 4
wait = 0.25
exp = 500
_old_exp=100
_old_bin = 4
_old_channel = 1
light_channel = 'cyan'
bins = 8
obj_60x = 0.0001927525057825752
conversion = obj_60x

def te300_presnap(ce_sys=ce_system, intensity=1):
    ce_system.excitation_wheel.set_intensity(['red','blue','green','cyan','uv','teal'],intensity)

def te300_postsnap(ce_sys=ce_system, intensity=0):
    ce_system.excitation_wheel.set_intensity(['red','blue','green','cyan','uv','teal'],intensity)

def pre_fluoresence(auto_shutter=False):
    # SNAP a FLUOR IMAGE
    # Set to new values
    ce_system.inlet_rgb.turn_off_channel(rgb_chnl)
    ce_system.camera.stop()
    ce_system.filter_wheel.set_channel(fluor_channel)
    ce_system.excitation_wheel.set_channel([light_channel])
    ce_system.camera.set_exposure(exp)
    #_old_exp = ce_system.camera.exposure
    ce_system.camera.set_binning(bins)
    #_old_bin = ce_system.camera.bin_size
    time.sleep(wait)
    
def post_fluoresence(auto_shutter=False):
    # Return to old values
    ce_system.camera.set_exposure(_old_exp)
    #ce_system.camera.set_binning(_old_bin)
    ce_system.filter_wheel.set_channel(_old_channel)
    ce_system.inlet_rgb.turn_on_channel(rgb_chnl)
    ce_system.excitation_wheel.set_channel([])
    ce_system.camera.set_binning(_old_bin)
    ce_system.camera.continuous_snap()

ce_system.camera._presnap_callbacks=[]
ce_system.camera._postsnap_callbacks=[]
ce_system.camera.add_presnap_callback(te300_presnap,ce_sys=ce_system)
ce_system.camera.add_postsnap_callback(te300_postsnap, ce_sys=ce_system)

# Find the Brightest Cell

In [26]:
pre_fluoresence()
img = ce_system.camera.snap()
post_fluoresence()
fig, ax = plt.subplots()
ax.imshow(img)

HEYYOO


In [15]:
ce_system.filter_wheel.set_channel(1)

In [32]:
ce_system.outlet_pressure.rinse_pressure()

In [31]:
ce_system.outlet_pressure.release()

In [16]:
ce_system.inlet_rgb.turn_on_channel('B')

'R:On, G:Off, B:On'

In [17]:
ce_system.inlet_rgb.turn_off_channel('R')

'R:Off, G:Off, B:On'

In [13]:
ce_system.excitation_wheel.set_channel(['cyan'])

In [19]:
ce_system.camera.snap()

hi there


array([[199, 203, 198, ..., 196, 199, 202],
       [201, 200, 199, ..., 200, 203, 196],
       [204, 202, 200, ..., 198, 196, 200],
       ...,
       [204, 199, 198, ..., 192, 195, 192],
       [201, 195, 197, ..., 195, 196, 197],
       [197, 201, 197, ..., 197, 196, 193]], dtype=uint16)

In [271]:
# Bring the Cells into focus
fluor_channel = 3
focus_plane.get_plane_focus()


In [236]:
# Search for the best focal plane near here
pre_fluoresence()
focus_search.search_step_global()
post_fluoresence()

HEYYOO


In [258]:
fluor_channel = 1
pre_fluoresence()
img = ce_system.camera.snap()
post_fluoresence()


HEYYOO


In [184]:
plt.figure()
plt.imshow(img)

In [259]:
close_size=5
open_size=5
watershed_footprint = (5,5)
# Make sure types are the same
input_image = img_as_float(img)

# Filter Image
filtered_image = filters.median(input_image, behavior='ndimage')

# Edge Detection
edge_sobel = filters.sobel(filtered_image)

# Threshold
thresh = filters.threshold_otsu(filtered_image)
binary_otsu = filtered_image > thresh

# Binary Morphology Operations
structure_element = morphology.disk(close_size)
closed_image = morphology.binary_closing(binary_otsu, structure_element)
structure_element = morphology.disk(open_size)
opened_image = morphology.binary_opening(closed_image, structure_element)

# Watershed
distance = distance_transform_edt(opened_image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
markers = label(local_maxi)[0]
labels = watershed(-distance, markers, mask=opened_image)

plt.figure()
plt.imshow(labels)

regions = regionprops_table(labels, intensity_image=img, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))


df = pd.DataFrame(regions)

df = df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)

In [260]:
if df.shape[0] > 0:
    y=df['centroid-0'][0]*bins
    x=df['centroid-1'][0]*bins
    
    movex = (x-target_x)*conversion
    movey = (y-target_y)*conversion
    ce_system.xy_stage.set_rel_x(movex)
    ce_system.xy_stage.set_rel_y(-movey)

# Single Cell Injection/Load 

In [54]:
focus_plane.get_plane_focus()

In [7]:
ce_system.filter_wheel.set_channel(1)

In [55]:
pre_fluoresence()
img = ce_system.camera.snap()
post_fluoresence()
fig, ax = plt.subplots()
ax.imshow(img)

HEYYOO


In [16]:
# on reruns, you just need to run this command when you have the cells in focus
resp = get_yes_no("are the cells in focus? (You may crush the capillary if not...)")
if resp:
    obj_height = ce_system.objective.read_z()
    print(obj_height)
    cap_z = get_cap_height(obj_height, cap_difference)
    z = ce_system.inlet_z.read_z()
    rel_z = cap_z-z
    ce_system.inlet_z.set_rel_z(rel_z)
else:
    print("Bring cells into focus.")

are the cells in focus? (You may crush the capillary if not...) y
-5.31


In [225]:
# Raise the objective for optimal plasma formation
ce_system.objective.set_rel_z(-.01)

In [17]:
# Lower the capillary (ADJUSTMENT)
ce_system.inlet_z.set_rel_z(-0.065)

In [51]:
# General Injection Parameters
injection_time = 4 # in seconds
gravity_drop = 0 # in millimeters
voltage_level = 3 # in kilovolts
delay = 0 # inseconds

# Start the Single Cell Injection
import threading

def limited_fire(ce_system, gravity_drop, voltage_level, injection_time):
    time.sleep(delay)
    print('Fire')
    ce_system.outlet_z.set_rel_z(-gravity_drop)
    ce_system.high_voltage.set_voltage(voltage_level)
    ce_system.high_voltage.start()
    ce_system.objective.set_rel_z(-0.02)
    start_time = time.time()
    while time.time()-start_time < injection_time:
        time.sleep(0.5)
        ce_system.objective.set_rel_z(0.005)
        ce_system.xy_stage.set_rel_x(0.001)
        #ce_system.lysis_laser.laser_fire()
    ce_system.high_voltage.stop()
    print(f"Finished {time.time()-start_time}")
    ce_system.outlet_z.set_rel_z(gravity_drop)
    

threading.Thread(target=limited_fire, args=(ce_system, gravity_drop, voltage_level, injection_time)).start()

In [22]:
ce_system.inlet_z.set_rel_z(3)

In [244]:
resp = ""
while resp != 'q':
    resp=input("Press enter to fire, press q to exit")
    if resp != 'q':
        ce_system.lysis_laser.laser_fire()

Press enter to fire, press q to exitq


In [53]:
# Start the Method
def go_ahead():
    ce_system.inlet_z.set_rel_z(3)
    time.sleep(2)
    ce_system.objective.set_rel_z(-6)
    if not auto_program.continue_event.is_set():
        auto_program.continue_event.set()
    else:
        auto_program.start_run()
threading.Thread(target=go_ahead).start()

After special True
Time to wait is  3


After special True
Time to wait is  150


In [ ]:
print('hi')

In [23]:
im = ImageSaver(ce_system,folder_prefix='degradation',data_folder=r"C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020")

In [27]:
im.remove_callback()


In [17]:
len(im.images)

100

In [13]:
im.remove_callback()

In [19]:
im = ImageSaver(ce_system,folder_prefix='degradation',data_folder=r"C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020")
def run_me():
    
    pre_fluoresence()
    im.add_callback()
    for i in range(100):
        img = ce_system.camera.snap()
        #fig, ax = plt.subplots()
        #ax.imshow(img)
    im.remove_callback()
    post_fluoresence()
    
threading.Thread(target=run_me).start()


HEYYOO


In [20]:
im.save_image(unique_folder=False)

C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00000.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00001.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00002.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00003.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_lau

C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00059.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00060.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00061.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00062.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykernel_launcher.py:42: UserWarning: C:\Users\NikonTE300CE\Desktop\Brae\9-24-2020\img_00063.tiff is a low contrast image
C:\Users\NikonTE300CE\Miniconda3\envs\CEpy37\lib\site-packages\ipykern

# PUT YOUR CRAP FUNCTIONS HERE

In [19]:
# Start Rinse
ce_system.outlet_pressure.rinse_pressure()

In [17]:
# Stop Rinse 
ce_system.outlet_pressure.release()

In [74]:
ce_system.inlet_z.set_z(22)

In [100]:
ce_system.objective.set_rel_z(-3)

After special True
Time to wait is  150


C:\Users\NikonTE300CE\Desktop\Barracuda\AutomatedCE\L4\DisplayWindows.py:277: UserWarning: Attempting to set identical left == right == 0.0 results in singular transformations; automatically expanding.
  ax1.set_xlim(min(data['time_data']), max(data['time_data']))


In [16]:
ce_system.outlet_pressure.rinse_vacuum()

In [18]:
ce_system.filter_wheel.set_channel(1)

In [87]:
ce_system.high_voltage.stop()

In [78]:
laser = ce_system.lysis_laser

In [292]:
laser.laser_standby()

In [291]:
laser.laser_stop()

In [293]:
laser.laser_fire()

In [11]:
ce_system.objective.set_rel_z(-3)

In [127]:
ce_system.inlet_rgb.turn_on_channel('R')


'R:On, G:Off, B:Off'

In [126]:
ce_system.inlet_rgb.turn_off_channel('R')


'R:Off, G:Off, B:Off'

In [58]:
xy = ce_system.xy_stage.read_xy()
xy

[28.52200042501092, -1.8983000282868743]

In [68]:
ce_system.xy_stage.set_xy(xy)

In [19]:
ce_system.xy_stage.set_rel_x(0.05)

In [20]:
191.5-61.8

129.7

In [21]:
129.7*2

259.4

In [22]:
0.05/259.4

0.0001927525057825752

In [10]:
ce_system.inlet_z.set_z(10)

In [75]:
ce_system.outlet_pressure.seal()

In [6]:
cam = ce_system.camera

In [7]:
ce_system.inlet_z.controller.open()

In [9]:
ce_system.inlet_z.startup()

In [3]:
output = r"C:\Users\Luke\Documents\Data\images"
im_sav = ImageSaver(ce_system, data_folder=output, prefix='testing')

In [4]:
im_sav.add_callback()

In [7]:
cam.continuous_snap()

HEYYOO


True

In [8]:
cam.stop()

True

In [9]:
im_sav.remove_callback()

In [14]:
im_sav.save_image(unique_folder=False)

C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00000.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00001.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00002.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\img_00003.tiff is a low contrast image
C:\Users\Luke\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:35: UserWarning: C:\Users\Luke\Documents\Data\images\i

In [2]:

class ImageSaver:
    
    def __init__(self, system:SystemsBuilder.CESystem, folder_prefix=None, data_folder=None):
        self.system = system
        self.prefix = folder_prefix
        self.data_folder = data_folder
        self.images = []
        self._lock = threading.RLock()
        
    def add_callback(self):
        self.system.camera.add_callback(self.callback, tag='save_img')
        
    def remove_callback(self):
        self.system.camera.remove_callback(tag='save_img')
    
    def callback(self, img, *args, **kwargs):
        with self._lock:
            self.images.append(img)
    
    def save_image(self, image_prefix='img', folder_prefix=None, data_folder=None, unique_folder=True):
        """
        Saves the images into an folder with incrementing image names:
        
        image_prefix: what the image filename should be caleld
        folder_prefix: what the data folder will be called
        data_folder: where the parent_directory resides
        unique_folder: When true, create a new folder everytime the function is called
        
        """
        
        if folder_prefix is None:
            prefix = self.prefix
        if data_folder is None:
            data_folder = self.data_folder
    
        if unique_folder:
            data_dir = FileIO.get_data_folder(prefix, data_folder)
        else:
            data_dir = data_folder
        for image in self.images:
            data_file = FileIO.get_data_filename(image_prefix, data_dir, extension='.tiff')
            io.imsave(data_file, image)
            
    
    